In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train= pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/train.csv')
test =pd.read_csv('/kaggle/input/tabular-playground-series-feb-2021/test.csv')

# 1. **import lib**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
test.head()

In [ ]:
train.describe()

## Check for null values 

In [ ]:
train.isnull().sum()

Split categorical and numerical variables

## Differentiate categorical and numerical columns

In [ ]:
train_x = train.drop(['id', 'target'], axis=1)

numerical = train[train_x.columns].select_dtypes(include=['int64','float64']).columns
categorical = train[train_x.columns].select_dtypes(exclude=['int64','float64']).columns

## Correlation function among Numerical column

In [ ]:
corr = train[numerical].corr().abs()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(14, 10))

sns.heatmap(corr, mask=mask, annot=True)

plt.show()

## Categorical column visualization

In [ ]:
def hist(x):
    
    sns.histplot(x=train[x], data=train)

    plt.show()
   
for i in range(0,10):
    
    hist('cat'+str(i))

## Numerical column visualization

In [ ]:
def kde(x):
    
    sns.kdeplot(x=train[x], color="m")

    plt.show()
   
for i in range(0,14):
    
    kde('cont'+str(i))

## Label encodeing categorical columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feature in categorical:
   
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])



In [ ]:
train.head()

## Defining train and test set to train model 

In [ ]:
from sklearn.model_selection import train_test_split
x = train_x
y = train['target']



x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Building h2o Model


# [h20 autoML Reference](http://www.geeksforgeeks.org/automl-using-h2o/)

In [ ]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

## split train-valid set

In [ ]:
train_hf = h2o.H2OFrame(train)
test_hf = h2o.H2OFrame(test)

predictors = list(train_x)
response = 'target'

# Split into train and test
train, valid = train_hf.split_frame(ratios=[.8], seed=1234)

# Training the model

In [ ]:
aml = H2OAutoML(
    max_models=20,
    max_runtime_secs=200,
    exclude_algos = ["DeepLearning", "DRF"],
    seed=42,
)
aml.train(x=predictors,
        y=response,
        training_frame=train,
        validation_frame=valid
)


## Model performance 

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

## Get the top model of leaderboard

In [ ]:
# Get the top model of leaderboard 
se = aml.leader 
rmse= aml.leader.rmse(valid=True)
mae = aml.leader.mae(valid=True)

print('The model performance in RMSE: {}'.format(rmse))
print('The model performance in MAE: {}'.format(mae))

In [ ]:
# Get the metalearner model of top model 
metalearner = h2o.get_model(se.metalearner().model_id)
metalearner.varimp()

## Feature importance

In [ ]:
model = h2o.get_model('GBM_grid__1_AutoML_20210219_060331_model_1')
model.varimp_plot(num_of_features = 10)

## Test set predicting

In [ ]:
preds = aml.predict(test_hf).as_data_frame()
preds.head()

In [ ]:
df=pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

# Submission file 1

In [ ]:
df['target'] = preds['predict']
df.to_csv('submission_h2o1.csv', index=False)
df.head()


# Work in progress